# Intent 와 NER 모델을 만들기 위한 Data의 구성 방법
- reuse of https://github.com/TensorMSA/tensormsa_jupyter/blob/master/chap13_chatbot_lecture/3.Chatbot(Data%20for%20Train).ipynb

 - 먹이질문, 훈련법질문, 질병질문, 성격(견종별)질문의 각각의 Entity구성 (서비스시에는 Entity는 별도 DB로 구성
 - Entity 별 N by N의 수로 정제된 Text를 구할 수 있음

In [1]:
train_data_feature = ['말티즈 성격 어때?']
train_data_sympt = ['자꾸 털이 빠지고 기침을 하네 ㅠㅠ']
train_data_vaccin = ['지금8개월인데예방접종뭐해야해?']
train_data_feed = ['초콜릿 먹여도 되냐?']

In [72]:
get_data_list = train_data_vaccin[0]

dict_entity = {
    'age' : ['개월', '살'],
    'feed' : ['초콜릿','사과','당근'],
    'menu' : ['피자','햄버거'],
    'vaccin' : ['접종', '예방', '주사'],
    'sympt' : ['털', '빠짐', '기침']
}

length = 1
for key in list(dict_entity.keys()):
    length = length * len(dict_entity[key])
print("Augmentation length is {0}".format(length))

Augmentation length is 108


In [4]:
get_data_list

'지금8개월인데예방접종뭐해야해?'

# 형태소 분석

In [3]:
from konlpy.tag import Mecab
mecab = Mecab('/usr/local/lib/mecab/dic/mecab-ko-dic')
morpphed_text = mecab.pos(get_data_list)

In [6]:
print(morpphed_text)

[('지금', 'MAG'), ('8', 'SN'), ('개월', 'NNBC'), ('인데', 'VCP+EC'), ('예방', 'NNG'), ('접종', 'NNG'), ('뭐', 'IC'), ('해야', 'VV+EC'), ('해', 'VX+EF'), ('?', 'SF')]


# Feature Engineering (명사만 도출)

Feature Engineering으로 Intent와 NER의 정확도를 높임)
 - 일반명사(NNG) [메뉴]
 - 고유명사(NNP) [지역]
 - 영어(SL) [Pizza]
 - 시간부사(MAG) [오늘, 내일, 모래], 시간단위(NNBC) [개월, 살]
 - 숫자(SN) 
 - 한국어 품사 태그 비교표 https://docs.google.com/spreadsheets/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0

In [73]:
tagged_text = ''
for pos_tags in morpphed_text:
    if (pos_tags[1] in ['NNG','MAG', 'NNP','SL', 'NNBC'] and len(pos_tags[0]) > 1): #Check only Noun
        feature_value = pos_tags[0]
        tagged_text = tagged_text + pos_tags[0] + ' '
print(tagged_text)

지금 개월 예방 접종 


In [74]:
tagged_num = []

for i in range(len(morpphed_text)-1):
    pos_tags = morpphed_text[i]
    if pos_tags[1] == 'SN': #Check only Noun
        feature_value = pos_tags[0] + morpphed_text[i+1][0]
        tagged_num.append(feature_value)
print(tagged_num)

['8개월']


In [86]:
# 'NNG','MAG', 'NNP','SL', 'NNBC' 지만, 예외인 단어들

except_list = ['지금', '이제']

# Intent 학습 Data의 구성

- Intent 성능 향상을 위해 parse한 Text Data를 represent화 함

In [87]:
pattern = ''
for word in tagged_text.split(' '):
    if word in except_list: continue
    entity = list(filter(lambda key:word in dict_entity[key],list(dict_entity.keys())))
    if(len(entity) > 0): 
        pattern = pattern + 'tag' + entity[0] + ' '
    else:
        pattern = pattern + word + ' '

print(pattern)

tagage tagvaccin tagvaccin  


# Data augmentation (Entity and Pattern)
 - 각 의도별 Pattern text를 entity의 N 배수로 Augmenatation 작업

In [88]:
def augmentation_pattern(pattern, dict_entity):
    #입력된 패턴을 List로 바꿈
    aug_pattern = pattern.split(' ')
    #Augment된 Text List
    augmented_text_list = []
    #copy를 위한 임시 List
    temp_aug = []
    for i in range(0,len(aug_pattern)):
        #Entity에 해당하는 값일 경우 Entity List를 가져옴
        if(aug_pattern[i].find("tag") > -1):
            dict_list = dict_entity[aug_pattern[i].replace("tag","")]
            #각 Entity별로 값을 append하면서 Pattern구성
            for j in range(0,len(dict_list)):
                #최초 Entity값은 그냥 추가만함
                if(i == 0):
                    augmented_text_list.append(dict_list[j] + " ")
                elif(j == 1):
                    augmented_text_list = list(filter(lambda word:len(word.split(' ')) == i + 1 ,augmented_text_list))
                    copy_data_order = augmented_text_list * (len(dict_list)-2)
                    augmented_text_list = list(map(lambda x:x + dict_list[j] + " ",augmented_text_list))
                    augmented_text_list = augmented_text_list + temp_aug + copy_data_order
                    print(augmented_text_list)
                else:
                    #List의 수를 체크하여 값을 추가
                    temp_aug = list(filter(lambda word:len(word.split(' ')) == i+1 ,augmented_text_list))
                    temp_aug = list(map(lambda x:x + dict_list[j] + " " ,temp_aug))
                    #추가된 List를 위해 기존 값 삭제
                    if(j != 0):
                        augmented_text_list = augmented_text_list[0:len(augmented_text_list) - len(temp_aug)]
                    augmented_text_list = augmented_text_list + temp_aug

        #Entity추가 대상이 아닐 경우 Pattern만 추가
        else:
            augmented_text_list = list(map(lambda x:x + aug_pattern[i] + " ",augmented_text_list))
        #N*N으로 증가시키기 위한 List
        temp_aug = augmented_text_list

    return augmented_text_list

In [89]:
augmented_text_list = augmentation_pattern(pattern, dict_entity)

['개월 예방 ', '살 예방 ', '개월 접종 ', '살 접종 ', '개월 ', '살 ']
['개월 예방 예방 ', '살 예방 예방 ', '개월 접종 예방 ', '살 접종 예방 ', '개월 주사 예방 ', '살 주사 예방 ', '개월 예방 접종 ', '살 예방 접종 ', '개월 접종 접종 ', '살 접종 접종 ', '개월 주사 접종 ', '살 주사 접종 ', '개월 예방 ', '살 예방 ', '개월 접종 ', '살 접종 ', '개월 주사 ', '살 주사 ']


In [90]:
augmented_text_list

['개월 예방 예방   ',
 '살 예방 예방   ',
 '개월 접종 예방   ',
 '살 접종 예방   ',
 '개월 주사 예방   ',
 '살 주사 예방   ',
 '개월 예방 접종   ',
 '살 예방 접종   ',
 '개월 접종 접종   ',
 '살 접종 접종   ',
 '개월 주사 접종   ',
 '살 주사 접종   ',
 '개월 예방 주사   ',
 '살 예방 주사   ',
 '개월 접종 주사   ',
 '살 접종 주사   ',
 '개월 주사 주사   ',
 '살 주사 주사   ']

# BIO Tagging

In [91]:
def augmentation_bio_pattern(pattern, dict_entity):
    #입력된 패턴을 List로 바꿈
    aug_pattern = pattern.split(' ')
    #Augment된 Text List
    augmented_text_list = []
    #copy를 위한 임시 List
    temp_aug = []
    for i in range(0,len(aug_pattern)):
        #Entity에 해당하는 값일 경우 Entity List를 가져옴
        if(aug_pattern[i].find("tag") > -1):
            dict_list = dict_entity[aug_pattern[i].replace("tag","")]
            bio_tag = aug_pattern[i].replace("tag","B_")
            #각 Entity별로 값을 append하면서 Pattern구성
            for j in range(0,len(dict_list)):
                #최초 Entity값은 그냥 추가만함
                if(i == 0):
                    augmented_text_list.append(bio_tag + " ")
                elif(j == 1):
                    augmented_text_list = list(filter(lambda word:len(word.split(' ')) == i + 1 ,augmented_text_list))
                    copy_data_order = augmented_text_list * (len(dict_list)-2)
                    augmented_text_list = list(map(lambda x:x + bio_tag + " ",augmented_text_list))
                    augmented_text_list = augmented_text_list + temp_aug + copy_data_order
                else:
                    #List의 수를 체크하여 값을 추가
                    temp_aug = list(filter(lambda word:len(word.split(' ')) == i+1 ,augmented_text_list))
                    temp_aug = list(map(lambda x:x + bio_tag + " " ,temp_aug))
                    #추가된 List를 위해 기존 값 삭제
                    if(j != 0):
                        augmented_text_list = augmented_text_list[0:len(augmented_text_list) - len(temp_aug)]
                    augmented_text_list = augmented_text_list + temp_aug

        #Entity추가 대상이 아닐 경우 Pattern만 추가
        else:
            augmented_text_list = list(map(lambda x:x + aug_pattern[i] + " ",augmented_text_list))
        #N*N으로 증가시키기 위한 List
        temp_aug = augmented_text_list
    return augmented_text_list

In [92]:
bio_list = augmentation_bio_pattern(pattern, dict_entity)
bio_list

['B_age B_vaccin B_vaccin   ',
 'B_age B_vaccin B_vaccin   ',
 'B_age B_vaccin B_vaccin   ',
 'B_age B_vaccin B_vaccin   ',
 'B_age B_vaccin B_vaccin   ',
 'B_age B_vaccin B_vaccin   ',
 'B_age B_vaccin B_vaccin   ',
 'B_age B_vaccin B_vaccin   ',
 'B_age B_vaccin B_vaccin   ',
 'B_age B_vaccin B_vaccin   ',
 'B_age B_vaccin B_vaccin   ',
 'B_age B_vaccin B_vaccin   ',
 'B_age B_vaccin B_vaccin   ',
 'B_age B_vaccin B_vaccin   ',
 'B_age B_vaccin B_vaccin   ',
 'B_age B_vaccin B_vaccin   ',
 'B_age B_vaccin B_vaccin   ',
 'B_age B_vaccin B_vaccin   ']

# NER을 위한 Full Train Text 확보
 - tag entity를 NER 학습을 위한 Labeled Entity로 변환 (Bi-LSTM 학습을 위함)

In [96]:
ner_train_text = [augmented_text_list, bio_list]
ner_train_text

[['개월 예방 예방   ',
  '살 예방 예방   ',
  '개월 접종 예방   ',
  '살 접종 예방   ',
  '개월 주사 예방   ',
  '살 주사 예방   ',
  '개월 예방 접종   ',
  '살 예방 접종   ',
  '개월 접종 접종   ',
  '살 접종 접종   ',
  '개월 주사 접종   ',
  '살 주사 접종   ',
  '개월 예방 주사   ',
  '살 예방 주사   ',
  '개월 접종 주사   ',
  '살 접종 주사   ',
  '개월 주사 주사   ',
  '살 주사 주사   '],
 ['B_age B_vaccin B_vaccin   ',
  'B_age B_vaccin B_vaccin   ',
  'B_age B_vaccin B_vaccin   ',
  'B_age B_vaccin B_vaccin   ',
  'B_age B_vaccin B_vaccin   ',
  'B_age B_vaccin B_vaccin   ',
  'B_age B_vaccin B_vaccin   ',
  'B_age B_vaccin B_vaccin   ',
  'B_age B_vaccin B_vaccin   ',
  'B_age B_vaccin B_vaccin   ',
  'B_age B_vaccin B_vaccin   ',
  'B_age B_vaccin B_vaccin   ',
  'B_age B_vaccin B_vaccin   ',
  'B_age B_vaccin B_vaccin   ',
  'B_age B_vaccin B_vaccin   ',
  'B_age B_vaccin B_vaccin   ',
  'B_age B_vaccin B_vaccin   ',
  'B_age B_vaccin B_vaccin   ']]

# 위에서 얻은 Data를 토데로 Intent와 NER의 모델을 얻기 위해 학습시킬 수 있음